In [29]:
pip install fake_useragent

Note: you may need to restart the kernel to use updated packages.


In [34]:
import requests
import random
from pprint import pprint
import sqlite3
from html import unescape
from bs4 import BeautifulSoup 
import re
import time
from fake_useragent import UserAgent
from pymorphy2 import MorphAnalyzer
import nltk
from nltk.tokenize import word_tokenize
import collections 

Я буду доставать отзывы на фильм Ларса фон Триера "Антихрист" с Кинопоиска. Кинопоиск позволяет открыть 200 отзывов сразу, поэтому нам не придется мучиться с доставанием отзывов из разных мест. С помощью Супа распарсю полученный html-файл, достав сначала все хорошие, а затем все плохие отзывы. Положу 35 первых хороших и 35 первых плохих в соответствующие списки. Также сразу сделаю тестовые выборки -- плохих и хороших.

In [35]:
def get_all(url_one):
    session = requests.session()
    ua = UserAgent(verify_ssl=False)
    req = session.get(url_one, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    good_reviews = soup.findAll('div', {'class': "response good"})
    goodie = []
    for i in good_reviews:
        i_1 = i.find('div', {'class': "brand_words"}).text.replace('\n', '').replace('\xa0', ' ').replace('\r', '')
        goodie.append(i_1)
    goodies = goodie[:35]
    g_test = goodie[35:40]
    soup = BeautifulSoup(page, 'html.parser')
    bad_reviews = soup.findAll('div', {'class': "response bad"})
    badie = []
    for i in bad_reviews:
        i_1 = i.find('div', {'class': "brand_words"}).text.replace('\n', '').replace('\xa0', ' ').replace('\r', '')
        badie.append(i_1)
    badies = badie[:35]
    b_test = badie[35:40]
    return(goodies, badies, g_test, b_test)

С помощью НЛТК и ПайМорфи токенизирую текст и приведу все слова к начальным формам.

In [36]:
def prep(goodies):
    prepr = []
    for i in goodies:
        text_py1 = []
        morph = MorphAnalyzer()
        text_py = [w.lower() for w in word_tokenize(i) if w.isalpha()]
        text_len = len(text_py)
        review = []
        for word in text_py:
            new_word = morph.parse(word)
            new_word1 = new_word[0]
            review.append(new_word1.normal_form)
        review = ' '.join(review)
        prepr.append(review)
    return prepr

Создам множества слов, которые есть только в хороших (а затем только в плохих) отзывах.

In [37]:
def get_unique(first, second):
    true = []
    all_f = ' '.join(first).split()
    all_s = ' '.join(second).split()
    for i in all_f:
        if i not in all_s:
            true.append(i)
    return true

Посчитаю частоту встречаемости каждого из слов в отзывах, уберу те слова, которые встречаются по одному разу. _(Я написала эту функцию, тк она была в задании, но с ней моя accuracy падает, видимо из-за маленькой выборки и красноречия авторов отзывов, поэтому я не буду ее использовать далее в самой программе. Но она рабочая и оставлена здесь, чтобы мне не снизили балл за невыполнение пункта)_

In [38]:
def get_commons(t):
    c = collections.Counter(t)
    commons = [word for word,v in c.items() if v > 1]
    return commons

Определю, хороший отзыв или плохой, посчитав, сколько в нем слов из хорошего и плохого набора. Если слов из хорошего набора больше -- считаем отзыв хорошим, если меньше -- плохим. Если равное количество -- относим его к противоречивым.

In [39]:
def good_or_bad(r, t_g, t_b):
    bads = 0
    goods = 0
    morph = MorphAnalyzer()
    text_py = [w.lower() for w in word_tokenize(r) if w.isalpha()]
    text_len = len(text_py)
    rev = []
    for word in text_py:
        new_word = morph.parse(word)
        new_word1 = new_word[0]
        rev.append(new_word1.normal_form)
    for i in rev:
        if i in t_b:
            bads += 1
        elif i in t_g:
            goods += 1
    if bads > goods:
        ton = 'bad'
    elif bads < goods:
        ton = 'good'
    else:
        ton = 'controversial'
    return ton

Самостоятельно посчитаю accuracy своей функции, разделив количество правильных ответов на общее количество отзывов в тестовой выборке.

In [40]:
def get_acc(test_g, test_b, g_un, b_un):
    acc = 0
    for review in test_g:
        if good_or_bad(review, g_un, b_un) == 'good':
            acc += 1
    for review in test_b:
        if good_or_bad(review, g_un, b_un) == 'bad':
            acc += 1
    acc_tot = acc/(len(test_g)+len(test_b))
    return acc_tot

Теперь запущу все полученные функции.

In [41]:
goodies1, badies1, g_test1, b_test1 = get_all('https://www.kinopoisk.ru/film/408909/reviews/ord/date/status/all/perpage/200/')
goodies2, badies2, g_test2, b_test2 = get_all('https://www.kinopoisk.ru/film/408909/reviews/ord/date/status/all/perpage/200/page/2/')
goodies = goodies1 + goodies2
badies = badies1 + badies2
g_test = g_test1 + g_test2
b_test = b_test1 + b_test2
g_prep = prep(goodies)
b_prep = prep(badies)
g_un = get_unique(g_prep, b_prep)
b_un = get_unique(b_prep, g_prep)

In [42]:
print('accuracy:', get_acc(g_test, b_test, g_un, b_un))

accuracy: 0.6


Чтобы увеличить accuracy, можно:
1. Взять бóльшую выборку отзывов, добавив другие фильмы того же режиссера (они схожей тематики, так что ключевые слова не будут сильно различаться)
2. Можно было бы воспользоваться word2vec, чтобы смотреть не только сами слова, встретившиеся в негативных/позитивных отзывах, но и синонимичные, близкие к ним по векторному представлению
3. Посмотреть не слова, а н-граммы 
4. Найти какую-то черту, отличающую положительные отзывы на этом сайте от отрицательных и смотреть по ней (например, возможно, авторы отрицательных отзывов более эмоциональны в своем гневе, тогда можно будет принимать во внимание знаки препинания).

## Здесь начинается дз_2

Копируем группирующую функцию из соседней тетради (пояснения см. там)

In [98]:
def get_groups(text):
    adj_noun = []
    no = []
    adv_p = []
    text = ''.join(ch for ch in text if ch not in exclude)
    i = re.split(r'\?|!|\.', text)
    for line in i:
        token = nltk.word_tokenize(line)
        bigram = list(ngrams(token, 2)) 
        for i in bigram:
            if morph.parse(i[0])[0].tag.POS == 'ADJF' and morph.parse(i[1])[0].tag.POS == 'NOUN':
                adj_noun.append(i)
            if i[0] == 'не' and morph.parse(i[1])[0].tag.POS == 'VERB':
                no.append(i)
            if morph.parse(i[0])[0].tag.POS == 'ADVB' and morph.parse(i[1])[0].tag.POS == 'VERB' or morph.parse(i[0])[0].tag.POS == 'VERB' and morph.parse(i[1])[0].tag.POS == 'ADVB':
                adv_p.append(i)
    return adj_noun, no, adv_p

С помощью пайморфи, нлтк и нашей функции получаем искомые группы сначала для хороших отзывов, затем для плохих.

In [52]:
from nltk import word_tokenize 
from nltk.util import ngrams
import re
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
exclude = [',','.', '!', '?', ':', '–', '(', ')']
exclude = set(exclude)
bi_g = []
for g in g_prep:
    NP = get_groups(g)[0]
    NO = get_groups(g)[1]
    ADV = get_groups(g)[2]
    for i in NP:
        bi_g.append(i)
    for i in NO:
        bi_g.append(i)
    for i in ADV:
        bi_g.append(i)

[('именитый', 'режиссёр'), ('литературный', 'часть'), ('литературный', 'часть'), ('актёрский', 'игра'), ('монтажный', 'склейка'), ('высокий', 'ценность'), ('литературный', 'ценность'), ('такой', 'режиссёр'), ('какой', 'место'), ('особый', 'место'), ('остальной', 'режиссёр'), ('какой', 'образ'), ('единственный', 'режиссёр'), ('широкий', 'публика'), ('талантливый', 'режиссёр'), ('данный', 'простота'), ('ложный', 'намёк'), ('красивый', 'композиция'), ('самобытный', 'саспенс'), ('главный', 'ценность'), ('литературный', 'часть'), ('данный', 'фильм'), ('сам', 'идея'), ('свой', 'отпечаток'), ('неординарный', 'звук'), ('мрачный', 'тон'), ('яркий', 'акцент'), ('задний', 'план'), ('единый', 'волна'), ('некоторый', 'фильм'), ('откровенный', 'цензура'), ('никакой', 'цензура'), ('этот', 'фильм'), ('откровенный', 'сцена'), ('откровенный', 'сцена'), ('настоящий', 'искусство'), ('чёрный', 'птица'), ('приятный', 'просмотр'), ('мрачный', 'фильм'), ('самый', 'носитель'), ('голый', 'тело'), ('интимный', '

In [53]:
bi_b = []
for g in b_prep:
    NP = get_groups(g)[0]
    NO = get_groups(g)[1]
    ADV = get_groups(g)[2]
    for i in NP:
        bi_b.append(i)
    for i in NO:
        bi_b.append(i)
    for i in ADV:
        bi_b.append(i)

[('скандальный', 'фильм'), ('этот', 'момент'), ('разный', 'идея'), ('начальный', 'сцена'), ('загородный', 'дом'), ('настоящий', 'анархия'), ('один', 'шаг'), ('весь', 'хронометраж'), ('весь', 'фильм'), ('эксцентрический', 'иносказание'), ('западноевропейский', 'эксцентричность'), ('западноевропейский', 'общество'), ('определённый', 'доза'), ('детский', 'упоение'), ('один', 'фильм'), ('свой', 'время'), ('талантливый', 'фильм'), ('финальный', 'план'), ('церковный', 'символика'), ('чужой', 'цитата'), ('чужой', 'андрей'), ('западноевропейский', 'потребитель'), ('самый', 'ларс'), ('иной', 'жанр'), ('интересный', 'объект'), ('пущий', 'медийность'), ('нужный', 'ларс'), ('такой', 'фильм'), ('сложный', 'отношение'), ('эпатажный', 'заявление'), ('этот', 'ужас'), ('страшный', 'ужас'), ('душевный', 'ужас'), ('такой', 'свойство'), ('весь', 'тематика'), ('западноевропейский', 'привычка'), ('свой', 'интервью'), ('подчинённый', 'положение'), ('сам', 'образ'), ('тихий', 'ужас'), ('западноевропейский', '

Как и в прошлый раз, ищем только уникально хорошие и уникально плохие биграммы.

In [55]:
def get_unique_b(all_f, all_s):
    true = []
    for i in all_f:
        if i not in all_s:
            true.append(i)
    return true

Решаем, хороший отзыв или плохой, используя и отдельные слова, и наши биграммы-синт.группы.

In [99]:
def good_or_bad_b(r, t_g, t_b, g, b):
    bads = 0
    goods = 0
    morph = MorphAnalyzer()
    text_py = [w.lower() for w in word_tokenize(r) if w.isalpha()]
    text_len = len(text_py)
    rev = []
    for word in text_py:
        new_word = morph.parse(word)
        new_word1 = new_word[0]
        rev.append(new_word1.normal_form)
    rev = ' '.join(rev)
    bis = []
    a, b, c = get_groups(rev)
    for i in a:
        bis.append(i)
    for i in b:
        bis.append(i)
    for i in c:
        bis.append(i)
    for i in bis:
        if i in bi_b:
            bads += 1
        elif i in bi_g:
            goods += 1
    for i in rev.split():
        if i in b:
            bads += 1
        elif i in g:
            goods += 1
    if bads > goods:
        ton = 'bad'
    elif bads < goods:
        ton = 'good'
    else:
        ton = 'controversial'
    return ton

Считаем accuracy.

In [92]:
def get_acc_b(test_g, test_b, g_un, b_un, g, b):
    acc = 0
    for review in test_g:
        if good_or_bad_b(review, g_un, b_un, g, b) == 'good':
            acc += 1
    for review in test_b:
        if good_or_bad_b(review, g_un, b_un, g, b) == 'bad':
            acc += 1
    acc_tot = acc/(len(test_g)+len(test_b))
    return acc_tot

In [97]:
true_good = get_unique_b(bi_g, bi_b)
true_bad = get_unique_b(bi_b, bi_g)

In [95]:
print('accuracy for new model:', get_acc_b(g_test, b_test, true_good, true_bad, g_un, b_un))

accuracy for new model: 0.55


In [96]:
print('accuracy for old model:', get_acc(g_test, b_test, g_un, b_un))

accuracy for old model: 0.6


Как ни странно, с использованием синт.групп наша accuracy немного, но понизилась. Возможно, это объясняется 1) красноречием авторов отзывов (они все же редко используют похожие биграммы) 2) Неоднозначностью фильмов Триера (я заметила, что комментаторы используют похожие хлесткие выражения и в хвалебных, и в ругательных отзывы